# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_reviews(mov_id, max_rev):
    rev = []
    current_page = 1
    while len(rev) < max_rev:
        url = f"https://www.imdb.com/title/{mov_id}/reviews/?ref_=tt_ql_2"
        params = {
            'spoiler': 'hide',
            'sort': 'helpfulnessScore',
            'dir': 'desc',
            'ratingFilter': '0',
            'paginationKey': '',
        }
        response = requests.get(url, params=params)
        soup = BeautifulSoup(response.content, 'html.parser')

        review_blocks = soup.find_all('div', class_='lister-item-content')
        if not review_blocks:
            break

        for block in review_blocks:
            title = block.find('a', class_='title').text.strip()
            review_text = block.find('div', class_='text show-more__control').text.strip()
            rev.append({'title': title, 'review': review_text})
            if len(rev) >= max_rev:
                break

        current_page += 1

    return rev[:max_rev]

def save_reviews_to_csv(rev, filename):
    df = pd.DataFrame(rev)
    df.to_csv(filename, index=False)

mov_id = 'tt15398776'
max_rev = 1000
rev = get_reviews(mov_id, max_rev)
save_reviews_to_csv(rev, 'reviews_movie.csv')


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [2]:

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load the data from CSV
df = pd.read_csv('/content/reviews_movie.csv')
print("Original Data:")
print(df.head())

#Step 1
df['clean_review'] = df['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

print(" Step 1: Remove noise, special characters, and punctuations:")
print(df[['title', 'clean_review']].head())

#step 2
df['clean_review'] = df['clean_review'].apply(lambda x: re.sub(r'\d+', '', x))
print("Step 2: Remove numbers:")
print(df[['title', 'clean_review']].head())

#Step 3
stop_words = set(stopwords.words('english'))
df['clean_review'] = df['clean_review'].apply(lambda x: ' '.join(word for word in x.split() if word.lower() not in stop_words))
print(" Step 3: Remove stopwords:")
print(df[['title', 'clean_review']].head())

#Step 4
df['clean_review'] = df['clean_review'].apply(lambda x: x.lower())
print("Step 4: Lowercase all texts:")
print(df[['title', 'clean_review']].head())

#Step 5
porter = PorterStemmer()
df['clean_review'] = df['clean_review'].apply(lambda x: ' '.join(porter.stem(word) for word in x.split()))
print("Step 5: Stemming:")
print(df[['title', 'clean_review']].head())

#Step 6
lemmatizer = WordNetLemmatizer()
df['clean_review'] = df['clean_review'].apply(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()))
print("Step 6: Lemmatization:")
print(df[['title', 'clean_review']].head())

df.to_csv('/content/reviews_movie.csv', index=False)
print("\nCleaned Data:")
print(df[['title', 'clean_review']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Original Data:
                                               title  \
0                              Murphy is exceptional   
1                             Quality but exhausting   
2  A brilliantly layered examination of a man thr...   
3  Nolan delivers a powerfull biopic that shows t...   
4                           And the Oscar goes to...   

                                              review  
0  You'll have to have your wits about you and yo...  
1  I'm a big fan of Nolan's work so was really lo...  
2  "Oppenheimer" is a biographical thriller film ...  
3  This movie is just... wow! I don't think I hav...  
4  I'm still collecting my thoughts after experie...  
 Step 1: Remove noise, special characters, and punctuations:
                                               title  \
0                              Murphy is exceptional   
1                             Quality but exhausting   
2  A brilliantly layered examination of a man thr...   
3  Nolan delivers a powerfull bio

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [3]:

import spacy
from spacy import displacy
from nltk import pos_tag, word_tokenize, RegexpParser
import nltk
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


df = pd.read_csv('/content/reviews_movie.csv')
print(df.head())

sentence1 = df['clean_review'].iloc[0]

# (1) Parts of Speech (POS) Tagging
def pos_tagging(text):
    doc = nlp(text)
    pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

    for token in doc:
        if token.pos_ == 'NOUN':
            pos_counts['Noun'] += 1
        elif token.pos_ == 'VERB':
            pos_counts['Verb'] += 1
        elif token.pos_ == 'ADJ':
            pos_counts['Adjective'] += 1
        elif token.pos_ == 'ADV':
            pos_counts['Adverb'] += 1

    return pos_counts

pos_result = pos_tagging(sentence1)
print("\n(1) Parts of Speech (POS) Tagging:")
print(pos_result)

# (2) Constituency Parsing and Dependency Parsing
doc = nlp(sentence1)

# Display Dependency Parsing Tree using spaCy
print("\nDependency Parsing Tree:")
displacy.render(doc, style="dep", options={'distance': 80})

# Tokenize and process the sentence with NLTK
words = word_tokenize(sentence1)
tagged_words = pos_tag(words)

# Define a simple grammar for NP (Noun Phrase) and VP (Verb Phrase)
grammar = r"""
    NP: {<DT>?<JJ>*<NN>}    # Noun Phrase
    VP: {<VB.*><NP|PP>}     # Verb Phrase
    PP: {<IN><NP>}          # Prepositional Phrase
"""

# Using RegexpParser from NLTK to perform constituency parsing
parser = RegexpParser(grammar)
tree = parser.parse(tagged_words)


print("\nConstituency Parsing Tree:")
tree.pretty_print()
#step 3
def named_entity_recognition(text):
    doc = nlp(text)
    entities = {}

    for ent in doc.ents:
        entities[ent.label_] = entities.get(ent.label_, 0) + 1

    return entities

ner_result = named_entity_recognition(sentence1)
print("\n(3) Named Entity Recognition (NER):")
print(ner_result)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


                                               title  \
0                              Murphy is exceptional   
1                             Quality but exhausting   
2  A brilliantly layered examination of a man thr...   
3  Nolan delivers a powerfull biopic that shows t...   
4                           And the Oscar goes to...   

                                              review  \
0  You'll have to have your wits about you and yo...   
1  I'm a big fan of Nolan's work so was really lo...   
2  "Oppenheimer" is a biographical thriller film ...   
3  This movie is just... wow! I don't think I hav...   
4  I'm still collecting my thoughts after experie...   

                                        clean_review  
0  youll wit brain fulli switch watch oppenheim c...  
1  im big fan nolan work realli look forward unde...  
2  oppenheim biograph thriller film written direc...  
3  movi wow dont think ever felt like watch movi ...  
4  im still collect thought experienc film cillia..


Constituency Parsing Tree:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         S                                                                                                                                                                                                                                                                                                                                  

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below

